# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

In [2]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, encoding = 'latin-1')

In [3]:
df.columns = ['target','id','date','flag','user','text']
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [4]:
def clean_up(s): 
    s = s.lower()
    s = re.sub("http:\S+", " ", s)
    s = re.findall("[a-z]+", s)
    return ' '.join(s)

def tokenize(s):
    return nltk.word_tokenize(s)


nltk.download('omw-1.4')

def stem_and_lemmatize(l):

    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]

    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in l]

    return lemmatized


def remove_stopwords(l):
    return list(set(l).difference(stopwords.words()))





[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kitchphil/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
df_sample = df.sample(20000)
#df_sample = df[0:20]
df_sample

,target,id,date,flag,user,text
637988,0,2234329299,Thu Jun 18 22:51:04 PDT 2009,NO_QUERY,duhitsmelinda,"@yayitsalma awh. yes, yes i did o_o i didnt e..."
1499042,4,2070726481,Sun Jun 07 18:11:40 PDT 2009,NO_QUERY,Vonnieee,@Thahani Yeah I was afraid he might lose since...
152959,0,1932855094,Tue May 26 22:22:42 PDT 2009,NO_QUERY,Byrdie05,I want this type of LOVE... lmao! L &amp; B l...
1595168,4,2192313964,Tue Jun 16 06:50:51 PDT 2009,NO_QUERY,deveronie,Gooooooood morning and happy Tuesday!!!! Xox f...
270942,0,1989931774,Mon Jun 01 03:18:47 PDT 2009,NO_QUERY,Jeff_Hardyfan,&quot;@mileycyrus&quot;you are the best!!!! I ...
...,...,...,...,...,...,...
201488,0,1971902665,Sat May 30 08:08:37 PDT 2009,NO_QUERY,Ernieeee,The train's delayed
500947,0,2187011888,Mon Jun 15 19:39:42 PDT 2009,NO_QUERY,mari_fletcher,@feltbeats nothing for brazil ? if tom comes ...
666448,0,2245288104,Fri Jun 19 15:55:20 PDT 2009,NO_QUERY,zaza97aj,"canne wait for Ugly Betty 2 start , although i..."
997015,4,1835991991,Mon May 18 07:09:41 PDT 2009,NO_QUERY,yaboon,bought a miss selfridge dress at half price b...


In [6]:
textcln = []
for i in df_sample['text']:
    textcln.append(remove_stopwords(stem_and_lemmatize(tokenize(clean_up(i)))))

In [7]:
txtjoin = []
for i in textcln:
    #i = " ".join(i)
    txtjoin.append(i)

In [8]:
df_sample['text_processed'] = txtjoin

In [9]:
df_sample

,target,id,date,flag,user,text,text_processed
637988,0,2234329299,Thu Jun 18 22:51:04 PDT 2009,NO_QUERY,duhitsmelinda,"@yayitsalma awh. yes, yes i did o_o i didnt e...","[nudge, clicked, happen, yayitsalma, awh, xd, ..."
1499042,4,2070726481,Sun Jun 07 18:11:40 PDT 2009,NO_QUERY,Vonnieee,@Thahani Yeah I was afraid he might lose since...,"[thahani, afraid, awesome, lose, wa, sd]"
152959,0,1932855094,Tue May 26 22:22:42 PDT 2009,NO_QUERY,Byrdie05,I want this type of LOVE... lmao! L &amp; B l...,"[type, love, lmao, luv, amp, b]"
1595168,4,2192313964,Tue Jun 16 06:50:51 PDT 2009,NO_QUERY,deveronie,Gooooooood morning and happy Tuesday!!!! Xox f...,"[morning, xox, gooooooood, happy, philly, tues..."
270942,0,1989931774,Mon Jun 01 03:18:47 PDT 2009,NO_QUERY,Jeff_Hardyfan,&quot;@mileycyrus&quot;you are the best!!!! I ...,"[mail, mileycyrus, quot, give, love, write]"
...,...,...,...,...,...,...,...
201488,0,1971902665,Sat May 30 08:08:37 PDT 2009,NO_QUERY,Ernieeee,The train's delayed,"[delayed, train]"
500947,0,2187011888,Mon Jun 15 19:39:42 PDT 2009,NO_QUERY,mari_fletcher,@feltbeats nothing for brazil ? if tom comes ...,"[feltbeats, gon, tom, brazil, haha, make, surp..."
666448,0,2245288104,Fri Jun 19 15:55:20 PDT 2009,NO_QUERY,zaza97aj,"canne wait for Ugly Betty 2 start , although i...","[leaving, scottish, canne, crushed, wait, ugly..."
997015,4,1835991991,Mon May 18 07:09:41 PDT 2009,NO_QUERY,yaboon,bought a miss selfridge dress at half price b...,"[girly, selfridge, gon, half, price, haha, mis..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
nl = []

for x in df_sample["text_processed"]:
    for i in x:
        nl.append(i)

In [11]:
newlist = nltk.FreqDist(nl)

wordlist = list(newlist.keys())[:5000]
wordlist

['nudge',
 'clicked',
 'happen',
 'yayitsalma',
 'awh',
 'xd',
 'ment',
 'thahani',
 'afraid',
 'awesome',
 'lose',
 'wa',
 'sd',
 'type',
 'love',
 'lmao',
 'luv',
 'amp',
 'b',
 'morning',
 'xox',
 'gooooooood',
 'happy',
 'philly',
 'tuesday',
 'mail',
 'mileycyrus',
 'quot',
 'give',
 'write',
 'plumber',
 'largely',
 'coming',
 'tomorrow',
 'pizza',
 'otoh',
 'kitchen',
 'excuse',
 'back',
 'forward',
 'commission',
 'sell',
 'gm',
 'chinese',
 'firm',
 'hummer',
 'sample',
 'yay',
 'watch',
 'center',
 'lip',
 'spotlight',
 'poutier',
 'fuller',
 'modernmom',
 'mouth',
 'gloss',
 'charm',
 'work',
 'world',
 'worse',
 'vocab',
 'maanmujunkie',
 'word',
 'remembering',
 'translate',
 'fail',
 'exam',
 'epically',
 'eng',
 'guess',
 'ger',
 'entered',
 'pakistan',
 'semi',
 'anna',
 'food',
 'championship',
 'rugby',
 'district',
 'yesterday',
 'zenjar',
 'made',
 'suck',
 'hater',
 'totally',
 'spell',
 'ive',
 'telling',
 'woo',
 'joshfittell',
 'reason',
 'cute',
 'babe',
 'love

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

In [12]:
def find_features(x):
    words = set(x)
    features = {}
    for i in wordlist:
        features[i] = (i in words)
    return features

In [13]:
featuresets = [(find_features(tweet), target) for (tweet, target) in list(zip(df_sample['text_processed'], df_sample['target']))]

print(len(featuresets))

20000


[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [14]:
train_set, test_set = featuresets[:2500], featuresets[2500:]


In [15]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [16]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.6725714285714286
Most Informative Features
                    sick = True                0 : 4      =     16.3 : 1.0
                    hate = True                0 : 4      =     14.1 : 1.0
               beautiful = True                4 : 0      =     11.4 : 1.0
                 welcome = True                4 : 0      =     10.1 : 1.0
                 missing = True                0 : 4      =      9.8 : 1.0
                  stupid = True                0 : 4      =      9.8 : 1.0
                    wish = True                0 : 4      =      8.6 : 1.0
                    hurt = True                0 : 4      =      8.5 : 1.0
                    lost = True                0 : 4      =      8.3 : 1.0
                headache = True                0 : 4      =      7.6 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [17]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [18]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [19]:
# your code here